In [5]:
file_path = "/home/vbertalan/Downloads/gpt2_data.json"

def read_first_line_of_json(file_path):
    """
    Reads and returns the first line from a JSON file.
    
    :param file_path: Path to the JSON file
    :return: The first line of the file as a string (including newline if present)
    """
    with open(file_path, 'r', encoding='utf-8') as json_file:
        first_line = json_file.readline()
    return first_line

read_first_line_of_json(file_path)

'key: /nfs/appdata/6500_build/rel/6500_16.90/290147/logs/_6500_build.cc/400G_otn_fmotr_simbc, value: [tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, mv cannot stat no such file or directory, sed can t read temp no such file or directory, error gnu compilers before are not supported anymore, error unsupported toolchain, no such file, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, tput no value for term and no t specified]\n'

In [7]:
import json

file_path = "/home/vbertalan/Downloads/gpt2_data.json"

def extract_value_string(file_path):
    """
    Reads the first 20 lines of a text file where each line has a format like:
    'key: ..., value: [ ... ]'
    and returns the string following 'value:' for each line, excluding any
    leading '[' and trailing ']'.
    
    :param file_path: Path to your file
    :return: List of strings (the part after 'value:' in each of the first 20 lines, 
             without the outer brackets)
    """
    extracted_values = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for _ in range(20):
            line = file.readline()
            if not line:
                # No more lines to read
                break
            
            # Remove any leading/trailing whitespace
            line = line.strip()
            
            # Split on "value:"
            if "value:" in line:
                parts = line.split("value:", 1)  # Split once, into [before_value, after_value]
                value_part = parts[1].strip()    # The substring after 'value:'
                
                # If the string starts with '[' and ends with ']', remove them
                if value_part.startswith("[") and value_part.endswith("]"):
                    value_part = value_part[1:-1].strip()
                
                extracted_values.append(value_part)
            else:
                # If the line doesn't contain "value:", append None or skip
                extracted_values.append(None)
    
    return extracted_values

mylist = extract_value_string(file_path)
print(mylist)

['tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, mv cannot stat no such file or directory, sed can t read temp no such file or directory, error gnu compilers before are not supported anymore, error unsupported toolchain, no such file, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, tput no value for term and no t specified', 'jq error at stdin null null has no keys, tput no value for term and no t specified, tput no value for term and no t specified, tput 

In [9]:
print(mylist[1])

jq error at stdin null null has no keys, tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, tput no value for term and no t specified, cp r not specified omitting directory, cp r not specified omitting directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory, cp cannot stat no such file or directory


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from tqdm import tqdm

# Step 1: Load GPT-2 tokenizer and add custom log templates
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

log_templates = [
    "Error encountered in module X",
    "Error encountered in",
    "Unexpected behavior in network communication",
    "System rebooted successfully",
    "Segmentation fault in memory allocation"
]

# Add log templates as tokens to the tokenizer
tokenizer.add_tokens(log_templates)

# Set padding token to EOS
tokenizer.pad_token = tokenizer.eos_token

# Step 2: Prepare log sequences (you may add your actual log data here)
sequences = [
    "Error encountered in module X The weather is great today. I am working hard.",
    "The system rebooted successfully after the error."
]

# Tokenize sequences
tokenized_sequences = tokenizer(
    sequences,  # List of sentences
    truncation=True,
    padding=True,  # Padding all sequences to the same length
    max_length=128,  # Set max length for input sequences
    return_tensors="pt"  # Return PyTorch tensors
)

# Step 3: Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Resize the model's embeddings to include the new tokens
model.resize_token_embeddings(len(tokenizer))

# Step 4: Prepare Dataset and DataLoader for training
class LogSequenceDataset(Dataset):
    def __init__(self, tokenized_sequences):
        self.input_ids = tokenized_sequences['input_ids']
        self.attention_mask = tokenized_sequences['attention_mask']
        
    def __len__(self):
        return self.input_ids.shape[0]
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

# Create the dataset and dataloader
dataset = LogSequenceDataset(tokenized_sequences)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Step 5: Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Step 6: Enable gradient updates on embeddings
model.get_input_embeddings().requires_grad_(True)

# Step 7: Train the model (Continual Pretraining)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3  # Adjust based on your training needs

for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)
    
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass (language modeling task)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Update the model's parameters
        optimizer.step()
        
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{epochs} completed.")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")

print("Fine-tuning completed and model saved.")


/home/vbertalan/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 1/1 [00:01<00:00,  1.02s/it, loss=99.5]


Epoch 1/3 completed.


100%|██████████| 1/1 [00:00<00:00,  1.50it/s, loss=84.8]


Epoch 2/3 completed.


100%|██████████| 1/1 [00:00<00:00,  1.64it/s, loss=75.8]


Epoch 3/3 completed.
Fine-tuning completed and model saved.
